# TimeCopilot: Foundation Model Ensemble

This notebook demonstrates the evaluation of a foundation model ensemble built using the [TimeCopilot](https://timecopilot.dev) library on the [GIFT-Eval](https://huggingface.co/spaces/Salesforce/GIFT-Eval) benchmark.

TimeCopilot is an open‑source AI agent for time series forecasting that provides a unified interface to multiple forecasting approaches, from foundation models to classical statistical, machine learning, and deep learning methods, along with built‑in ensemble capabilities for robust and explainable forecasting.



## Model Description

This ensemble leverages [**TimeCopilot's MedianEnsemble**](https://timecopilot.dev/api/models/ensembles/#timecopilot.models.ensembles.median.MedianEnsemble) feature, which combines five state-of-the-art foundation models:

- [**Chronos** (Amazon)](https://timecopilot.dev/api/models/foundational/models/#timecopilot.models.foundational.chronos.Chronos).
- [**Moirai** (Salesforce)](https://timecopilot.dev/api/models/foundational/models/#timecopilot.models.foundational.moirai.Moirai).
- [**TimesFM** (Google)](https://timecopilot.dev/api/models/foundational/models/#timecopilot.models.foundational.timesfm.TimesFM).
- [**TiRex** (NX-AI)](https://timecopilot.dev/api/models/foundational/models/#timecopilot.models.foundational.tirex.TiRex).
- [**Toto** (DataDog)](https://timecopilot.dev/api/models/foundational/models/#timecopilot.models.foundational.toto.Toto).

The ensemble uses **median aggregation with isotonic regression** to ensure monotonic quantiles for probabilistic forecasting, providing robustness against outliers and model-specific biases.

## TimeCopilot's Key Features

- [**Foundation model integration**](https://timecopilot.dev/model_hub/): Unified API for 6+ state‑of‑the‑art foundation models
- **Ensemble capabilities**: Built-in ensemble methods
- **Zero-shot capability**: Leverages pretrained foundation models out‑of‑the‑box
- **Dependency management**: Handles complex model requirements automatically
- **GPU efficiency**: Optimized memory sharing and multi‑model execution

## Requirements and Installation

Install TimeCopilot library:


In [ ]:
%pip install "timecopilot>=0.0.13"

## Dataset Setup

TimeCopilot includes built-in [GIFT-Eval integration](https://timecopilot.dev/api/gift_eval/gift_eval/#timecopilot.gift_eval.eval.GIFTEval) for dataset handling:

In [ ]:
from timecopilot.gift_eval.eval import GIFTEval

# TimeCopilot's built-in GIFT-Eval dataset downloader
# Handles the complete benchmark dataset with all 97 configurations
storage_path = "./data/gift-eval"
GIFTEval.download_data(storage_path=storage_path)

## Model Implementation

Using TimeCopilot's [model hub](https://timecopilot.dev/model_hub/) and [ensemble capabilities](https://timecopilot.dev/api/models/ensembles/) to create a foundation model ensemble:

In [2]:
from timecopilot.models.ensembles.median import MedianEnsemble
from timecopilot.models.foundational.chronos import Chronos
from timecopilot.models.foundational.moirai import Moirai
from timecopilot.models.foundational.timesfm import TimesFM
from timecopilot.models.foundational.tirex import TiRex
from timecopilot.models.foundational.toto import Toto
from timecopilot.models.utils.forecaster import Forecaster

batch_size = 64

# TimeCopilot's MedianEnsemble with isotonic regression for robust forecasting
# Automatically handles dependency conflicts and GPU memory management
ensemble = MedianEnsemble(
    models=[
        # Each model uses TimeCopilot's unified interface despite different architectures
        Chronos(
            repo_id="amazon/chronos-bolt-base",
            batch_size=batch_size,
        ),
        Moirai(
            repo_id="Salesforce/moirai-1.1-R-base",
            batch_size=batch_size,
        ),
        TimesFM(batch_size=batch_size),
        TiRex(batch_size=batch_size),
        Toto(
            context_length=256,
            batch_size=batch_size,
        ),
    ],
    alias="TimeCopilot",
)

## Evaluation

### Defining the evaluator

With TimeCopilot you can evaluate any [Forecaster](https://timecopilot.dev/api/models/utils/forecaster/#timecopilot.models.utils.forecaster.Forecaster) in a standardized way using its [GIFT-Eval](https://timecopilot.dev/api/gift_eval/gift_eval/#timecopilot.gift_eval.eval.GIFTEval) integration.

In [3]:
import pandas as pd
from timecopilot.gift_eval.eval import GIFTEval
from timecopilot.gift_eval.gluonts_predictor import GluonTSPredictor


def evaluate_forecaster(
        forecaster: Forecaster,
        dataset_name: str,
        term: str,
        output_path: str,
        storage_path: str,
    ):
    """Evaluate a forecaster on a GIFT-Eval dataset defined by dataset name and term."""

    # TimeCopilot's GIFT-Eval loader handles dataset preprocessing automatically
    gifteval = GIFTEval(
        dataset_name=dataset_name,
        term=term,
        output_path=output_path,
        storage_path=storage_path,
    )

    # GluonTS wrapper for GIFT-Eval compatibility
    # It can receive any Forecaster from TimeCopilot
    predictor = GluonTSPredictor(
        forecaster=forecaster,
        max_length=4_096,
        batch_size=1_024,
    )

    # Run evaluation with GIFT-Eval's standardized metrics
    gifteval.evaluate_predictor(predictor, batch_size=512)

### Performing evaluation

In the GIFT-Eval benchmark, each dataset is defined by a combination of a dataset name and its term (short, medium or long).

In [ ]:
combinations = [
    ("m4_weekly", "short"),
    ("bizitobs_l2c/H", "short"),
    ("bizitobs_l2c/H", "medium"),
    ("bizitobs_l2c/H", "long"),
]

for dataset_name, term in combinations:
    evaluate_forecaster(
        forecaster=ensemble,
        dataset_name=dataset_name,
        term=term,
        output_path=f"./results/timecopilot",
        storage_path=storage_path,
    )

# Load consolidated results in GIFT-Eval format
eval_df = pd.read_csv("./results/timecopilot/all_results.csv")


In [5]:
eval_df

,dataset,model,eval_metrics/MSE[mean],eval_metrics/MSE[0.5],eval_metrics/MAE[0.5],eval_metrics/MASE[0.5],eval_metrics/MAPE[0.5],eval_metrics/sMAPE[0.5],eval_metrics/MSIS,eval_metrics/RMSE[mean],eval_metrics/NRMSE[mean],eval_metrics/ND[0.5],eval_metrics/mean_weighted_sum_quantile_loss,domain,num_variates
0,m4_weekly/W/short,TimeCopilot,247374.434877,247374.434877,248.012029,1.877632,0.059364,0.059972,14.214837,497.367505,0.090613,0.045184,0.035980,Econ/Fin,1
1,bizitobs_l2c/H/short,TimeCopilot,107.521631,107.521631,6.317921,0.640324,0.475619,0.716102,4.182129,10.369264,0.558930,0.340552,0.266380,Web/CloudOps,7
2,bizitobs_l2c/H/medium,TimeCopilot,127.658752,127.658752,6.958818,0.706064,0.689723,0.843889,4.710453,11.298617,0.684148,0.421367,0.327091,Web/CloudOps,7
3,bizitobs_l2c/H/long,TimeCopilot,117.121714,117.121714,6.837732,0.717908,0.754906,0.854900,5.215012,10.822279,0.661056,0.417668,0.325968,Web/CloudOps,7


You can access the complete combination of datasets with the following:

In [6]:
from timecopilot.gift_eval.utils import DATASETS_WITH_TERMS

In [7]:
DATASETS_WITH_TERMS[:3]

[('m4_yearly', 'short'), ('m4_quarterly', 'short'), ('m4_monthly', 'short')]

In [16]:
len(DATASETS_WITH_TERMS)

97

The code for the complete evaluation can be found in the [library's repo](https://github.com/AzulGarza/timecopilot/tree/main/experiments/gift-eval/).

## Reproducibility statement

The TimeCopilot's [GIFT-Eval integration](https://timecopilot.dev/api/gift_eval/gift_eval/#timecopilot.gift_eval.eval.GIFTEval) was designed considering reproducibility as one of its main features. The library can replicate the official results provided by the mantainers of the benchmark for the [`SeasonalNaive`](https://huggingface.co/spaces/Salesforce/GIFT-Eval/tree/main/results/seasonal_naive) method. The following code replicates the Seasonal Naive performance for the datasets evaluated in this notebook. The reproducibility of the results for the rest of the datasets are tested continuously in the [library's repo](https://github.com/AzulGarza/timecopilot/blob/main/tests/gift_eval/test_evaluation.py).

In [ ]:
from timecopilot.models.benchmarks import SeasonalNaive

combinations = [
    ("m4_weekly", "short"),
    ("bizitobs_l2c/H", "short"),
    ("bizitobs_l2c/H", "medium"),
    ("bizitobs_l2c/H", "long"),
]

for dataset_name, term in combinations:
    evaluate_forecaster(
        forecaster=SeasonalNaive(alias="Seasonal_Naive"),
        dataset_name=dataset_name,
        term=term,
        output_path=f"./results/seasonal_naive",
        storage_path=storage_path,
    )
eval_df_sn = pd.read_csv("./results/seasonal_naive/all_results.csv")

In [14]:
eval_df_sn

,dataset,model,eval_metrics/MSE[mean],eval_metrics/MSE[0.5],eval_metrics/MAE[0.5],eval_metrics/MASE[0.5],eval_metrics/MAPE[0.5],eval_metrics/sMAPE[0.5],eval_metrics/MSIS,eval_metrics/RMSE[mean],eval_metrics/NRMSE[mean],eval_metrics/ND[0.5],eval_metrics/mean_weighted_sum_quantile_loss,domain,num_variates
0,m4_weekly/W/short,Seasonal_Naive,453525.145918,453525.145918,347.991483,2.777295,0.089373,0.091613,26.631225,673.442756,0.122691,0.063399,0.060870,Econ/Fin,1
1,bizitobs_l2c/H/short,Seasonal_Naive,281.843068,281.843068,12.531653,1.214064,1.360590,1.138373,7.486931,16.788182,0.904926,0.675488,0.521168,Web/CloudOps,7
2,bizitobs_l2c/H/medium,Seasonal_Naive,456.373289,456.373289,15.667392,1.510286,1.691291,1.402410,18.533654,21.362895,1.293556,0.948684,0.904205,Web/CloudOps,7
3,bizitobs_l2c/H/long,Seasonal_Naive,309.272222,309.272222,13.635488,1.426054,2.438311,0.916854,22.036198,17.586137,1.074212,0.832895,0.941065,Web/CloudOps,7


In [10]:
official_eval_sn = pd.read_csv(
    "https://huggingface.co/spaces/Salesforce/GIFT-Eval/raw/main/results/seasonal_naive/all_results.csv"
)

In [11]:
official_eval_sn = official_eval_sn.set_index("dataset").loc[eval_df_sn["dataset"]].reset_index()

In [15]:
official_eval_sn

,dataset,model,eval_metrics/MSE[mean],eval_metrics/MSE[0.5],eval_metrics/MAE[0.5],eval_metrics/MASE[0.5],eval_metrics/MAPE[0.5],eval_metrics/sMAPE[0.5],eval_metrics/MSIS,eval_metrics/RMSE[mean],eval_metrics/NRMSE[mean],eval_metrics/ND[0.5],eval_metrics/mean_weighted_sum_quantile_loss,domain,num_variates
0,m4_weekly/W/short,Seasonal_Naive,453525.145918,453525.145918,347.991483,2.777295,0.089373,0.091613,26.631225,673.442756,0.122691,0.063399,0.060870,Econ/Fin,1
1,bizitobs_l2c/H/short,Seasonal_Naive,281.843068,281.843068,12.531653,1.214064,1.360590,1.138373,7.486931,16.788182,0.904926,0.675488,0.521168,Web/CloudOps,7
2,bizitobs_l2c/H/medium,Seasonal_Naive,456.373289,456.373289,15.667392,1.510286,1.691291,1.402410,18.533654,21.362895,1.293556,0.948684,0.904205,Web/CloudOps,7
3,bizitobs_l2c/H/long,Seasonal_Naive,309.272222,309.272222,13.635488,1.426054,2.438311,0.916854,22.036198,17.586137,1.074212,0.832895,0.941065,Web/CloudOps,7


In [12]:
pd.testing.assert_frame_equal(official_eval_sn, eval_df_sn)